In [3]:
import sys, os

sys.path.append(
    os.path.dirname(
        os.path.abspath(
            ""
        )
    )
)





In [5]:
from statements_list import Statements_list

S = Statements_list()

list_of_tickers = ["AAPL", "MSFT"]

In [7]:
S.generate_statement_list_multi(list_of_tickers, 'balance-sheet', 'annual')

ValueError: No objects to concatenate